In [1]:
from pyspark.sql import SparkSession
from pyspark import SparkContext
from pyspark import SparkConf
from pyspark.sql import functions as f
from pyspark.sql.functions import col, lit, length, substring, expr
from pyspark.sql import DataFrame

spark=SparkSession.builder.appName("Dunhumbby Practise").getOrCreate()
rawDF=spark.read.format("CSV").option("header","true").option("inferSchema","true").option("delimiter",",").load("/FileStore/tables/AviationData.csv")
rawDF.printSchema()

root
-- Event_Id: string (nullable = true)
-- Investigation_Type: string (nullable = true)
-- Accident_Number: string (nullable = true)
-- Event_Date: timestamp (nullable = true)
-- Location: string (nullable = true)
-- Country: string (nullable = true)
-- Latitude: double (nullable = true)
-- Longitude: double (nullable = true)
-- Airport_Code: string (nullable = true)
-- Airport_Name: string (nullable = true)
-- Injury_Severity: string (nullable = true)
-- Aircraft_Damage: string (nullable = true)
-- Aircraft_Category: string (nullable = true)
-- Registration_Number: string (nullable = true)
-- Make: string (nullable = true)
-- Model: string (nullable = true)
-- Amateur_Built: string (nullable = true)
-- Number_of_Engines: integer (nullable = true)
-- Engine_Type: string (nullable = true)
-- FAR_Description: string (nullable = true)
-- Schedule: string (nullable = true)
-- Purpose_of_Flight: string (nullable = true)
-- Air_Carrier: string (nullable = true)
-- Total_Fatal_Injuries: integer (nullable = true)
-- Total_Serious_Injuries: integer (nullable = true)
-- Total_Minor_Injuries: integer (nullable = true)
-- Total_Uninjured: integer (nullable = true)
-- Weather_Condition: string (nullable = true)
-- Broad_Phase_of_Flight: string (nullable = true)
-- Report_Status: string (nullable = true)
-- Publication_Date: timestamp (nullable = true)

In [2]:
#all the analysis will be done for United States
rawDF=rawDF.filter(rawDF["Country"]=="United States")
rawDF=rawDF.withColumn("State", col("Location").substr( -2 , 2 ))
#f.length(f.expr("Location"))-2 , f.length(f.expr("Location")) 
rawDF.select("State","Location").show(2)
rawDF.persist()

+-----+------------------+
State| Location|
+-----+------------------+
 CA|San Bernardino, CA|
 SD| Spearfish, SD|
+-----+------------------+
only showing top 2 rows

 Out[ 51 ]: DataFrame[Event_Id: string, Investigation_Type: string, Accident_Number: string, Event_Date: timestamp, Location: string, Country: string, Latitude: double, Longitude: double, Airport_Code: string, Airport_Name: string, Injury_Severity: string, Aircraft_Damage: string, Aircraft_Category: string, Registration_Number: string, Make: string, Model: string, Amateur_Built: string, Number_of_Engines: int, Engine_Type: string, FAR_Description: string, Schedule: string, Purpose_of_Flight: string, Air_Carrier: string, Total_Fatal_Injuries: int, Total_Serious_Injuries: int, Total_Minor_Injuries: int, Total_Uninjured: int, Weather_Condition: string, Broad_Phase_of_Flight: string, Report_Status: string, Publication_Date: timestamp, State: string]

In [3]:
#which month is least safe to travel.
monthDF=rawDF.withColumn("Event_Month", f.month("Event_Date"))
monthDF.groupBy("Event_Month").agg(f.count("Event_Month").alias("month_count")).sort(col("month_count").desc()).limit(1).show()


+-----------+-----------+
Event_Month|month_count|
+-----------+-----------+
 7| 9304|
+-----------+-----------+

In [4]:
#The amateur have a influence on accident.
#If the total incident count difference in percent is greater than 20% they have influence.
valid_amateurDF = rawDF.filter(col("Amateur_Built")!="")
DF_count = valid_amateurDF.count()
grouped_DF = valid_amateurDF.groupBy("Amateur_Built").agg(f.count("Amateur_Built").alias("unq_amateur_cnt"))
grouped_DF.withColumn("percent_influence", (col("unq_amateur_cnt")/DF_count)*100 ).show()

+-------------+---------------+------------------+
Amateur_Built|unq_amateur_cnt| percent_influence|
+-------------+---------------+------------------+
 No| 68319| 89.85676894950744|
 Yes| 7712|10.143231050492563|
+-------------+---------------+------------------+

In [5]:
#How many month they usually take to create the report.
publishedDF=rawDF.filter(f.length(col("Publication_Date"))>0).withColumn("days_for_rprt", f.datediff(col("Publication_Date"), col("Event_Date")))
publishedDF.agg(f.rint(f.avg("days_for_rprt")).alias("avg_days_for_report")).show()

+-------------------+
avg_days_for_report|
+-------------------+
 453.0|
+-------------------+

In [6]:
#What is the main reason for fatal error 
#     +++  OR +++
#which phase security check need to be reviewed and need time to time drills.
fatal_accident_DF = rawDF.filter( col("Injury_Severity").contains("Fatal(") & col("Broad_Phase_of_Flight").isNotNull() )
fatal_accident_DF.groupBy("Broad_Phase_of_Flight").agg(f.count("Broad_Phase_of_Flight").alias("phase_count")).sort(col("phase_count").desc()).limit(5).show()

+---------------------+-----------+
Broad_Phase_of_Flight|phase_count|
+---------------------+-----------+
 MANEUVERING| 3838|
 CRUISE| 2869|
 TAKEOFF| 2318|
 APPROACH| 1931|
 CLIMB| 684|
+---------------------+-----------+

In [7]:
#which state saw maximum number of accidents each year
rawDF.groupBy("State").agg(f.count("Event_Id").alias("event_cnt_per_state")).sort(col("event_cnt_per_state").desc()).limit(5).show()


+-----+-------------------+
State|event_cnt_per_state|
+-----+-------------------+
 CA| 8354|
 TX| 5366|
 FL| 5342|
 AK| 5254|
 AZ| 2607|
+-----+-------------------+

In [8]:
#which company badly need to review their quality check process.
rawDF.filter( col("Make")!="" ).groupBy("Make").agg(f.count("Event_Id").alias("maker_count")).sort(col("maker_count").desc()).limit(5).show()

+------+-----------+
 Make|maker_count|
+------+-----------+
CESSNA| 17030|
 PIPER| 9340|
Cessna| 7300|
 Piper| 3872|
 BEECH| 3089|
+------+-----------+

In [9]:
#top 3 airport in each state which saw maximum number of accident.
from pyspark.sql import Window
countByStateByAirport = rawDF.dropna(subset=["State","Airport_Code"]).groupBy("State","Airport_Code").agg(f.count("Event_Id").alias("accident_count"))
#countByStateByAirport.sort(col("State")).show(10)
window = Window.partitionBy(col("State")).orderBy(col("accident_count").desc())
countByStateByAirport.select("State","Airport_Code","accident_count",f.row_number().over(window).alias("rank")).filter(col("rank")<4).show(50)

+-----+------------+--------------+----+
State|Airport_Code|accident_count|rank|
+-----+------------+--------------+----+
 AZ| FFZ| 90| 1|
 AZ| PRC| 81| 2|
 AZ| DVT| 80| 3|
 SC| CRE| 33| 1|
 SC| CAE| 29| 2|
 SC| NONE| 24| 3|
 LA| NONE| 53| 1|
 LA| NEW| 34| 2|
 LA| BTR| 17| 3|
 MN| FCM| 47| 1|
 MN| MSP| 33| 2|
 MN| MIC| 28| 3|
 NJ| EWR| 51| 1|
 NJ| CDW| 43| 2|
 NJ| TEB| 43| 3|
 DC| DCA| 17| 1|
 DC| IAD| 10| 2|
 DC| KDCA| 2| 3|
 OR| HIO| 38| 1|
 OR| PDX| 26| 2|
 OR| 7S3| 21| 3|
 VA| NONE| 30| 1|
 VA| IAD| 29| 2|
 VA| HEF| 27| 3|
 RI| BID| 23| 1|
 RI| WST| 18| 2|
 RI| SFZ| 16| 3|
 KY| CVG| 29| 1|
 KY| LOU| 28| 2|
 KY| LEX| 21| 3|
 WY| RKS| 29| 1|
 WY| CPR| 28| 2|
 WY| RWL| 25| 3|
 NH| ASH| 18| 1|
 NH| 7B3| 16| 2|
 NH| LCI| 16| 3|
 MI| PTK| 51| 1|
 MI| DTW| 44| 2|
 MI| NONE| 30| 3|
 NV| VGT| 79| 1|
 NV| LAS| 51| 2|
 NV| RNO| 50| 3|
 WI| OSH| 94| 1|
 WI| MKE| 35| 2|
 WI| ENW| 23| 3|
 ID| BOI| 43| 1|
 ID| SUN| 29| 2|
 ID| IDA| 26| 3|
 CA| VNY| 92| 1|
 CA| LGB| 81| 2|
+-----+------------+--------------+----+
only showing top 50 rows